In [ ]:
class Gadget:
    def __init__(self, id: int, name: str) -> None:
        self.id = id
        self.name = name

    def use(self) -> None:
        print(f"Using {self.name} with Id#{self.id}")

    def __repr__(self) -> str:
        return f"Gadget(id={self.id}, name={self.name!r})"
    
    def __str__(self) -> str:
        return f"Gadget({self.id}, {self.name})"

In [ ]:
g1 = Gadget(1, "ipad")

In [ ]:
g1.use()

In [ ]:
id(g1)

In [ ]:
type(g1)

In [ ]:
eval(repr(g1))

In [ ]:
str(g1)

# Klasy